In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from decouple import config
import time
import pandas as pd 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options



# Inicializar el driver de Selenium con las opciones definidas
driver = webdriver.Chrome()

# Ir a la página de inicio de LinkedIn
driver.get("https://www.bumeran.com.pe/login?returnTo=/")


user =config("USER_LKN")
clave =config("PASS_LKN")

# Encontrar el campo de inicio de sesión y enviar credenciales
username = driver.find_element(By.ID,"email")
username.send_keys(user)
password = driver.find_element(By.ID,"password")
password.send_keys(clave)

time.sleep(3)

boton1=driver.find_element(By.XPATH,'//a[text()=" Acepto"]')
boton1.click()


# Hacer clic en el botón de inicio de sesión
boton=driver.find_element(By.ID,"ingresar")
boton.click()

time.sleep(10)

# Inicializar la lista de datos
datas = []

for i in range(0, 5):

    # Construir la URL
    if i == 0:
        url = 'https://www.bumeran.com.pe/empleos-busqueda-data.html'
    else:
        url = f'https://www.bumeran.com.pe/empleos-busqueda-data.html?page={i+1}'
    
    # Navegar a la URL
    driver.get(url)
    time.sleep(5)

    # Encontrar la tabla en la página

    table = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, 'sc-kdPYzB.egIfIy')))

    table_rows = table.find_elements(By.TAG_NAME, 'div') 
    print(f"Found {len(table_rows)} rows on the table from page {i}")

    tablex = []



    # Iterar a través de las filas de la tabla
    for row in table_rows:
        # Usar los selectores CSS correctos para su sitio web
        title_elements = row.find_elements(By.CLASS_NAME,"sc-fhiYOA.zGAkU") 
        company_elements = row.find_elements(By.CLASS_NAME,"sc-flvzOl.kJsfXU")
        location_elements = row.find_elements(By.CLASS_NAME,'sc-cRULEh.dNmbcO')
        link_elements = row.find_elements(By.CLASS_NAME, 'sc-jkPxnQ.ioakMb')

        title = title_elements[0].text if title_elements else None
        company = company_elements[0].text if company_elements else None
        location = location_elements[0].text if location_elements else None


        # Extraer el texto de los elementos encontrados
        title = title_elements[0].text if title_elements else None
        link = link_elements[0].get_attribute('href') if link_elements else None
        # Imprimir los detalles de cada fila
        print(f'Title: {title}, company: {company}, location: {location},Link: {link}')

        # Agregar los datos a la lista
        tablex.append({"title": title, "company": company, "location": location, "Link": link})

    # Convertir la lista en un DataFrame de pandas y agregarlo a la lista de datas
    df = pd.DataFrame(tablex)
    datas.append(df)

# Cerrar el driver de Selenium
#driver.quit()


Found 501 rows on the table from page 0
Title: Data Engineer, company: NTT DATA, location: Chorrillos, Lima,Link: https://www.bumeran.com.pe/empleos/data-engineer-ntt-data-1115960702.html
Title: Data Engineer, company: NTT DATA, location: Chorrillos, Lima,Link: None
Title: None, company: None, location: None,Link: None
Title: Data Engineer, company: NTT DATA, location: None,Link: None
Title: None, company: NTT DATA, location: None,Link: None
Title: None, company: NTT DATA, location: None,Link: None
Title: None, company: NTT DATA, location: None,Link: None
Title: None, company: None, location: None,Link: None
Title: None, company: NTT DATA, location: None,Link: None
Title: None, company: NTT DATA, location: None,Link: None
Title: None, company: -, location: None,Link: None
Title: Data Engineer, company: None, location: None,Link: None
Title: Data Engineer, company: None, location: None,Link: None
Title: Data Engineer, company: None, location: None,Link: None
Title: None, company: None, 

In [4]:
final_df = pd.concat(datas, ignore_index=True)
final_df

,title,company,location,Link
0,Data Engineer,NTT DATA,"Chorrillos, Lima",https://www.bumeran.com.pe/empleos/data-engine...
1,Data Engineer,NTT DATA,"Chorrillos, Lima",None
2,None,None,None,None
3,Data Engineer,NTT DATA,None,None
4,None,NTT DATA,None,None
...,...,...,...,...
2432,None,None,None,None
2433,None,None,None,None
2434,None,None,None,None
2435,None,None,None,None


In [5]:
final_df = final_df[final_df['Link'].str.len() > 5]


In [6]:
# Asumiendo que final_df ya está creado
palabras_clave = [ 
    "Datos",
    "Analista", 
    "Big Data", 
    "Scientist",
    "Científico",
    "Azure", 
    "Analyst", 
    "Analista",
    "engineer",
    "ingeniero"
]


palabras_excluir = [
    "Administrador", 
    "Arquitecto", 
    "Subgerente"
]


regex_incluir = '|'.join(palabras_clave)
regex_excluir = '|'.join(palabras_excluir)

final_df_filtrado = final_df[final_df['title'].str.contains(regex_incluir, case=False, na=False)]
final_df_filtrado = final_df_filtrado[~final_df_filtrado['title'].str.contains(regex_excluir, case=False, na=False)]


In [7]:
final_df_filtrado.to_csv("name2.csv")

In [9]:
final_df_filtrado

,title,company,location,Link
0,Data Engineer,NTT DATA,"Chorrillos, Lima",https://www.bumeran.com.pe/empleos/data-engine...
35,Data Engineer Azure,NTT DATA,"Lima, Lima",https://www.bumeran.com.pe/empleos/data-engine...
61,Data Engineer Big Data,Zoluxiones SAC,"Surquillo, Lima",https://www.bumeran.com.pe/empleos/data-engine...
87,DATA ENGINEER,Cencosud Perú S.A,"Miraflores, Lima",https://www.bumeran.com.pe/empleos/data-engine...
110,Data Engineer,Zoluxiones SAC,"Miraflores, Lima",https://www.bumeran.com.pe/empleos/data-engine...
...,...,...,...,...
2193,Data Engineer - Banca y finanzas,NEORIS PERU,"Lima, Lima",https://www.bumeran.com.pe/empleos/data-engine...
2217,Data Engineer Semi Senior - Azure,Bluetab Solutions,Perú,https://www.bumeran.com.pe/empleos/data-engine...
2243,Operador de Centro de Datos,Win Empresas,"Santiago de Surco, Lima",https://www.bumeran.com.pe/empleos/operador-de...
2265,ANALISTA DE REPORTES/DATOS - JUNIOR,Grupo Fractalia,"Surquillo, Lima",https://www.bumeran.com.pe/empleos/analista-de...


In [8]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Asumiendo que final_df_filtrado ya está creado

# Crea un nuevo libro de trabajo y una hoja de trabajo
wb = Workbook()
ws = wb.active

# Transfiere los datos del DataFrame a la hoja de trabajo, creando hiperlinks para la columna de links
for i, row in enumerate(dataframe_to_rows(final_df_filtrado, index=False, header=True)):
    row = list(row)
    if i == 0:
        ws.append(row)
    else:
        ws.append(row[:-1])
        ws['D%d' % (i+1)].hyperlink = row[-1]
        ws['D%d' % (i+1)].style = 'Hyperlink'

# Guarda el archivo
wb.save('final_df_filtrado.xlsx')
